In [1]:
# encoding: UTF-8
# Copyright 2017 Google.com
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn  # rnn stuff temporarily in contrib, moving back to code in TF 1.1
import os
import time
import math
import numpy as np
import my_txtutils as txt
tf.set_random_seed(0)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

# model parameters
#
# Usage:
#   Training only:
#         Leave all the parameters as they are
#         Disable validation to run a bit faster (set validation=False below)
#         You can follow progress in Tensorboard: tensorboard --log-dir=log
#   Training and experimentation (default):
#         Keep validation enabled
#         You can now play with the parameters anf follow the effects in Tensorboard
#         A good choice of parameters ensures that the testing and validation curves stay close
#         To see the curves drift apart ("overfitting") try to use an insufficient amount of
#         training data (shakedir = "shakespeare/t*.txt" for example)
#
SEQLEN = 30
BATCHSIZE = 200
ALPHASIZE = txt.ALPHASIZE
INTERNALSIZE = 512
NLAYERS = 3
learning_rate = 0.001  # fixed learning rate
dropout_pkeep = 0.8    # some dropout

In [3]:
# encoding: UTF-8
# Copyright 2017 Google.com
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import glob
import sys

# size of the alphabet that we work with
ALPHASIZE = 98


# Specification of the supported alphabet (subset of ASCII-7)
# 10 line feed LF
# 32-64 numbers and punctuation
# 65-90 upper-case letters
# 91-97 more punctuation
# 97-122 lower-case letters
# 123-126 more punctuation
def convert_from_alphabet(a):
    """Encode a character
    :param a: one character
    :return: the encoded value
    """
    if a == 9:
        return 1
    if a == 10:
        return 127 - 30  # LF
    elif 32 <= a <= 126:
        return a - 30
    else:
        return 0  # unknown


# encoded values:
# unknown = 0
# tab = 1
# space = 2
# all chars from 32 to 126 = c-30
# LF mapped to 127-30
def convert_to_alphabet(c, avoid_tab_and_lf=False):
    """Decode a code point
    :param c: code point
    :param avoid_tab_and_lf: if True, tab and line feed characters are replaced by '\'
    :return: decoded character
    """
    if c == 1:
        return 32 if avoid_tab_and_lf else 9  # space instead of TAB
    if c == 127 - 30:
        return 92 if avoid_tab_and_lf else 10  # \ instead of LF
    if 32 <= c + 30 <= 126:
        return c + 30
    else:
        return 0  # unknown


def encode_text(s):
    """Encode a string.
    :param s: a text string
    :return: encoded list of code points
    """
    return list(map(lambda a: convert_from_alphabet(ord(a)), s))


def decode_to_text(c, avoid_tab_and_lf=False):
    """Decode an encoded string.
    :param c: encoded list of code points
    :param avoid_tab_and_lf: if True, tab and line feed characters are replaced by '\'
    :return:
    """
    return "".join(map(lambda a: chr(convert_to_alphabet(a, avoid_tab_and_lf)), c))


def sample_from_probabilities(probabilities, topn=ALPHASIZE):
    """Roll the dice to produce a random integer in the [0..ALPHASIZE] range,
    according to the provided probabilities. If topn is specified, only the
    topn highest probabilities are taken into account.
    :param probabilities: a list of size ALPHASIZE with individual probabilities
    :param topn: the number of highest probabilities to consider. Defaults to all of them.
    :return: a random integer
    """
    p = np.squeeze(probabilities)
    p[np.argsort(p)[:-topn]] = 0
    p = p / np.sum(p)
    return np.random.choice(ALPHASIZE, 1, p=p)[0]


def rnn_minibatch_sequencer(raw_data, batch_size, sequence_size, nb_epochs):
    """
    Divides the data into batches of sequences so that all the sequences in one batch
    continue in the next batch. This is a generator that will keep returning batches
    until the input data has been seen nb_epochs times. Sequences are continued even
    between epochs, apart from one, the one corresponding to the end of raw_data.
    The remainder at the end of raw_data that does not fit in an full batch is ignored.
    :param raw_data: the training text
    :param batch_size: the size of a training minibatch
    :param sequence_size: the unroll size of the RNN
    :param nb_epochs: number of epochs to train on
    :return:
        x: one batch of training sequences
        y: on batch of target sequences, i.e. training sequences shifted by 1
        epoch: the current epoch number (starting at 0)
    """
    data = np.array(raw_data)
    data_len = data.shape[0]
    # using (data_len-1) because we must provide for the sequence shifted by 1 too
    nb_batches = (data_len - 1) // (batch_size * sequence_size)
    assert nb_batches > 0, "Not enough data, even for a single batch. Try using a smaller batch_size."
    rounded_data_len = nb_batches * batch_size * sequence_size
    xdata = np.reshape(data[0:rounded_data_len], [batch_size, nb_batches * sequence_size])
    ydata = np.reshape(data[1:rounded_data_len + 1], [batch_size, nb_batches * sequence_size])

    for epoch in range(nb_epochs):
        for batch in range(nb_batches):
            x = xdata[:, batch * sequence_size:(batch + 1) * sequence_size]
            y = ydata[:, batch * sequence_size:(batch + 1) * sequence_size]
            x = np.roll(x, -epoch, axis=0)  # to continue the text from epoch to epoch (do not reset rnn state!)
            y = np.roll(y, -epoch, axis=0)
            yield x, y, epoch


def find_book(index, bookranges):
    return next(
        book["name"] for book in bookranges if (book["start"] <= index < book["end"]))


def find_book_index(index, bookranges):
    return next(
        i for i, book in enumerate(bookranges) if (book["start"] <= index < book["end"]))


def print_learning_learned_comparison(X, Y, losses, bookranges, batch_loss, batch_accuracy, epoch_size, index, epoch):
    """Display utility for printing learning statistics"""
    print()
    # epoch_size in number of batches
    batch_size = X.shape[0]  # batch_size in number of sequences
    sequence_len = X.shape[1]  # sequence_len in number of characters
    start_index_in_epoch = index % (epoch_size * batch_size * sequence_len)
    for k in range(batch_size):
        index_in_epoch = index % (epoch_size * batch_size * sequence_len)
        decx = decode_to_text(X[k], avoid_tab_and_lf=True)
        decy = decode_to_text(Y[k], avoid_tab_and_lf=True)
        bookname = find_book(index_in_epoch, bookranges)
        formatted_bookname = "{: <10.40}".format(bookname)  # min 10 and max 40 chars
        epoch_string = "{:4d}".format(index) + " (epoch {}) ".format(epoch)
        loss_string = "loss: {:.5f}".format(losses[k])
        print_string = epoch_string + formatted_bookname + " │ {} │ {} │ {}"
        print(print_string.format(decx, decy, loss_string))
        index += sequence_len
    # box formatting characters:
    # │ \u2502
    # ─ \u2500
    # └ \u2514
    # ┘ \u2518
    # ┴ \u2534
    # ┌ \u250C
    # ┐ \u2510
    format_string = "└{:─^" + str(len(epoch_string)) + "}"
    format_string += "{:─^" + str(len(formatted_bookname)) + "}"
    format_string += "┴{:─^" + str(len(decx) + 2) + "}"
    format_string += "┴{:─^" + str(len(decy) + 2) + "}"
    format_string += "┴{:─^" + str(len(loss_string)) + "}┘"
    footer = format_string.format('INDEX', 'BOOK NAME', 'TRAINING SEQUENCE', 'PREDICTED SEQUENCE', 'LOSS')
    print(footer)
    # print statistics
    batch_index = start_index_in_epoch // (batch_size * sequence_len)
    batch_string = "batch {}/{} in epoch {},".format(batch_index, epoch_size, epoch)
    stats = "{: <28} batch loss: {:.5f}, batch accuracy: {:.5f}".format(batch_string, batch_loss, batch_accuracy)
    print()
    print("TRAINING STATS: {}".format(stats))


class Progress:
    """Text mode progress bar.
    Usage:
            p = Progress(30)
            p.step()
            p.step()
            p.step(start=True) # to restart form 0%
    The progress bar displays a new header at each restart."""
    def __init__(self, maxi, size=100, msg=""):
        """
        :param maxi: the number of steps required to reach 100%
        :param size: the number of characters taken on the screen by the progress bar
        :param msg: the message displayed in the header of the progress bat
        """
        self.maxi = maxi
        self.p = self.__start_progress(maxi)()  # () to get the iterator from the generator
        self.header_printed = False
        self.msg = msg
        self.size = size

    def step(self, reset=False):
        if reset:
            self.__init__(self.maxi, self.size, self.msg)
        if not self.header_printed:
            self.__print_header()
        next(self.p)

    def __print_header(self):
        print()
        format_string = "0%{: ^" + str(self.size - 6) + "}100%"
        print(format_string.format(self.msg))
        self.header_printed = True

    def __start_progress(self, maxi):
        def print_progress():
            # Bresenham's algorithm. Yields the number of dots printed.
            # This will always print 100 dots in max invocations.
            dx = maxi
            dy = self.size
            d = dy - dx
            for x in range(maxi):
                k = 0
                while d >= 0:
                    print('=', end="", flush=True)
                    k += 1
                    d -= dx
                d += dy
                yield k

        return print_progress


def read_data_files(directory, validation=True):
    """Read data files according to the specified glob pattern
    Optionnaly set aside the last file as validation data.
    No validation data is returned if there are 5 files or less.
    :param directory: for example "data/*.txt"
    :param validation: if True (default), sets the last file aside as validation data
    :return: training data, validation data, list of loaded file names with ranges
     If validation is
    """
    codetext = []
    bookranges = []
    shakelist = glob.glob(directory, recursive=True)
    for shakefile in shakelist:
        shaketext = open(shakefile, "r")
        print("Loading file " + shakefile)
        start = len(codetext)
        codetext.extend(encode_text(shaketext.read()))
        end = len(codetext)
        bookranges.append({"start": start, "end": end, "name": shakefile.rsplit("/", 1)[-1]})
        shaketext.close()

    if len(bookranges) == 0:
        sys.exit("No training data has been found. Aborting.")

    # For validation, use roughly 90K of text,
    # but no more than 10% of the entire text
    # and no more than 1 book in 5 => no validation at all for 5 files or fewer.

    # 10% of the text is how many files ?
    total_len = len(codetext)
    validation_len = 0
    nb_books1 = 0
    for book in reversed(bookranges):
        validation_len += book["end"]-book["start"]
        nb_books1 += 1
        if validation_len > total_len // 10:
            break

    # 90K of text is how many books ?
    validation_len = 0
    nb_books2 = 0
    for book in reversed(bookranges):
        validation_len += book["end"]-book["start"]
        nb_books2 += 1
        if validation_len > 90*1024:
            break

    # 20% of the books is how many books ?
    nb_books3 = len(bookranges) // 5

    # pick the smallest
    nb_books = min(nb_books1, nb_books2, nb_books3)

    if nb_books == 0 or not validation:
        cutoff = len(codetext)
    else:
        cutoff = bookranges[-nb_books]["start"]
    valitext = codetext[cutoff:]
    codetext = codetext[:cutoff]
    return codetext, valitext, bookranges


def print_data_stats(datalen, valilen, epoch_size):
    datalen_mb = datalen/1024.0/1024.0
    valilen_kb = valilen/1024.0
    print("Training text size is {:.2f}MB with {:.2f}KB set aside for validation.".format(datalen_mb, valilen_kb)
          + " There will be {} batches per epoch".format(epoch_size))


def print_validation_header(validation_start, bookranges):
    bookindex = find_book_index(validation_start, bookranges)
    books = ''
    for i in range(bookindex, len(bookranges)):
        books += bookranges[i]["name"]
        if i < len(bookranges)-1:
            books += ", "
    print("{: <60}".format("Validating on " + books), flush=True)


def print_validation_stats(loss, accuracy):
    print("VALIDATION STATS:                                  loss: {:.5f},       accuracy: {:.5f}".format(loss,
                                                                                                           accuracy))


def print_text_generation_header():
    print()
    print("┌{:─^111}┐".format('Generating random text from learned state'))


def print_text_generation_footer():
    print()
    print("└{:─^111}┘".format('End of generation'))


def frequency_limiter(n, multiple=1, modulo=0):
    def limit(i):
        return i % (multiple * n) == modulo*multiple
    return limit


In [4]:
# load data, either shakespeare, or the Python source of Tensorflow itself
shakedir = "txts/*.txt"
#shakedir = "../tensorflow/**/*.py"
codetext, valitext, bookranges = read_data_files(shakedir, validation=True)

Loading file txts/matt02_new.txt
Loading file txts/gal_eph_new.txt
Loading file txts/tit_phl_new.txt
Loading file txts/john_new.txt
Loading file txts/matt_new.txt
Loading file txts/heb_new.txt
Loading file txts/acts_new.txt
Loading file txts/jud_rev_new.txt
Loading file txts/luke_8_john_new.txt
Loading file txts/rom_new.txt
Loading file txts/mark01_new.txt
Loading file txts/jam_jud_new.txt
Loading file txts/phil_col_new.txt
Loading file txts/thes_tim_new.txt


In [5]:
# display some stats on the data
epoch_size = len(codetext) // (BATCHSIZE * SEQLEN)
print_data_stats(len(codetext), len(valitext), epoch_size)

Training text size is 3.14MB with 156.63KB set aside for validation. There will be 548 batches per epoch


In [6]:
#
# the model (see FAQ in README.md)
#
lr = tf.placeholder(tf.float32, name='lr')  # learning rate
pkeep = tf.placeholder(tf.float32, name='pkeep')  # dropout parameter
batchsize = tf.placeholder(tf.int32, name='batchsize')


In [7]:
# inputs
X = tf.placeholder(tf.uint8, [None, None], name='X')    # [ BATCHSIZE, SEQLEN ]
Xo = tf.one_hot(X, ALPHASIZE, 1.0, 0.0)                 # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
# expected outputs = same sequence shifted by 1 since we are trying to predict the next character
Y_ = tf.placeholder(tf.uint8, [None, None], name='Y_')  # [ BATCHSIZE, SEQLEN ]
Yo_ = tf.one_hot(Y_, ALPHASIZE, 1.0, 0.0)               # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
# input state
Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name='Hin')  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

# using a NLAYERS=3 layers of GRU cells, unrolled SEQLEN=30 times
# dynamic_rnn infers SEQLEN from the size of the inputs Xo

# How to properly apply dropout in RNNs: see README.md
cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]
# "naive dropout" implementation
dropcells = [rnn.DropoutWrapper(cell,input_keep_prob=pkeep) for cell in cells]
multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = rnn.DropoutWrapper(multicell, output_keep_prob=pkeep)  # dropout for the softmax layer

Yr, H = tf.nn.dynamic_rnn(multicell, Xo, dtype=tf.float32, initial_state=Hin)
# Yr: [ BATCHSIZE, SEQLEN, INTERNALSIZE ]
# H:  [ BATCHSIZE, INTERNALSIZE*NLAYERS ] # this is the last state in the sequence

H = tf.identity(H, name='H')  # just to give it a name

In [8]:
# Softmax layer implementation:
# Flatten the first two dimension of the output [ BATCHSIZE, SEQLEN, ALPHASIZE ] => [ BATCHSIZE x SEQLEN, ALPHASIZE ]
# then apply softmax readout layer. This way, the weights and biases are shared across unrolled time steps.
# From the readout point of view, a value coming from a sequence time step or a minibatch item is the same thing.

Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
Ylogits = layers.linear(Yflat, ALPHASIZE)     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Yflat_ = tf.reshape(Yo_, [-1, ALPHASIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]
Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE, SEQLEN ]
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

In [9]:
# stats for display
seqloss = tf.reduce_mean(loss, 1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))
loss_summary = tf.summary.scalar("batch_loss", batchloss)
acc_summary = tf.summary.scalar("batch_accuracy", accuracy)
summaries = tf.summary.merge([loss_summary, acc_summary])

# Init Tensorboard stuff. This will save Tensorboard information into a different
# folder at each run named 'log/<timestamp>/'. Two sets of data are saved so that
# you can compare training and validation curves visually in Tensorboard.
timestamp = str(math.trunc(time.time()))
summary_writer = tf.summary.FileWriter("log/" + timestamp + "-training")
validation_writer = tf.summary.FileWriter("log/" + timestamp + "-validation")

# Init for saving models. They will be saved into a directory named 'checkpoints'.
# Only the last checkpoint is kept.
if not os.path.exists("checkpoints"):
    os.mkdir("checkpoints")
saver = tf.train.Saver(max_to_keep=1000)

In [ ]:
# for display: init the progress bar
DISPLAY_FREQ = 50
_50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
progress = txt.Progress(DISPLAY_FREQ, size=111+2, msg="Training on next "+str(DISPLAY_FREQ)+" batches")

# init
istate = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])  # initial zero input state
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
step = 0

In [ ]:
# training loop
for x, y_, epoch in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=10):

    # train on one minibatch
    feed_dict = {X: x, Y_: y_, Hin: istate, lr: learning_rate, pkeep: dropout_pkeep, batchsize: BATCHSIZE}
    _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)

    # log training data for Tensorboard display a mini-batch of sequences (every 50 batches)
    if step % _50_BATCHES == 0:
        feed_dict = {X: x, Y_: y_, Hin: istate, pkeep: 1.0, batchsize: BATCHSIZE}  # no dropout for validation
        y, l, bl, acc, smm = sess.run([Y, seqloss, batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_learning_learned_comparison(x, y, l, bookranges, bl, acc, epoch_size, step, epoch)
        summary_writer.add_summary(smm, step)

    # run a validation step every 50 batches
    # The validation text should be a single sequence but that's too slow (1s per 1024 chars!),
    # so we cut it up and batch the pieces (slightly inaccurate)
    # tested: validating with 5K sequences instead of 1K is only slightly more accurate, but a lot slower.
    if step % _50_BATCHES == 0 and len(valitext) > 0:
        VALI_SEQLEN = 1*1024  # Sequence length for validation. State will be wrong at the start of each sequence.
        bsize = len(valitext) // VALI_SEQLEN
        txt.print_validation_header(len(codetext), bookranges)
        vali_x, vali_y, _ = next(txt.rnn_minibatch_sequencer(valitext, bsize, VALI_SEQLEN, 1))  # all data in 1 batch
        vali_nullstate = np.zeros([bsize, INTERNALSIZE*NLAYERS])
        feed_dict = {X: vali_x, Y_: vali_y, Hin: vali_nullstate, pkeep: 1.0,  # no dropout for validation
                     batchsize: bsize}
        ls, acc, smm = sess.run([batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_validation_stats(ls, acc)
        # save validation data for Tensorboard
        validation_writer.add_summary(smm, step)

    # display a short text generated with the current weights and biases (every 150 batches)
    if step // 3 % _50_BATCHES == 0:
        txt.print_text_generation_header()
        ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
        rh = np.zeros([1, INTERNALSIZE * NLAYERS])
        for k in range(1000):
            ryo, rh = sess.run([Yo, H], feed_dict={X: ry, pkeep: 1.0, Hin: rh, batchsize: 1})
            rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
            print(chr(txt.convert_to_alphabet(rc)), end="")
            ry = np.array([[rc]])
        txt.print_text_generation_footer()

    # save a checkpoint (every 500 batches)
    if step // 10 % _50_BATCHES == 0:
        saved_file = saver.save(sess, 'checkpoints/rnn_train_' + timestamp, global_step=step)
        print("Saved file: " + saved_file)

    # display progress bar
    progress.step(reset=step % _50_BATCHES == 0)

    # loop state around
    istate = ostate
    step += BATCHSIZE * SEQLEN


   0 (epoch 0) matt02_new.txt │  matio     nyo koruapu ma igbi │                                │ loss: 4.41082
  30 (epoch 0) matt02_new.txt │ i-e)  jizos nyengi ma na oria  │                                │ loss: 4.43337
  60 (epoch 0) matt02_new.txt │ ikidikibo oforika anana simeok │                                │ loss: 4.45615
  90 (epoch 0) matt02_new.txt │  o seni fini bie gbin na, piki │                                │ loss: 4.37775
 120 (epoch 0) matt02_new.txt │ ma ori ora toku be nwose okoap │                                │ loss: 4.43986
 150 (epoch 0) matt02_new.txt │ o siki, jizos be o viniki so k │                                │ loss: 4.42336
 180 (epoch 0) matt02_new.txt │ . aninakaraka o nwose o beme,  │                                │ loss: 4.42386
 210 (epoch 0) matt02_new.txt │ ia obuduoapuma   gbelame, nde  │                                │ loss: 4.45890
 240 (epoch 0) matt02_new.txt │ iki iya erebara anga sime.  ji │                                │ loss:

VALIDATION STATS:                                  loss: 4.28746,       accuracy: 0.19547

┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
nX}oKmee}IonoaonE ababFam  oib amb} e ebomb ibieieoiioiiwb nm  i mi    nmmbbebm nobkaaikanw nbnmbooa inomeanoemabnoebaaii  wibb}n Feminobmmmamme}nibbe}nb  e}b  bwema}wn oea} mieoebi}en m e no }koiiaabmbem nm wo mbn n}} ommanboemnnniXb ommbie}eb}bibai}Fmbambebebnn } e }wnik kbao}} }ai}eebmoininnemXibnebooi}o}FnaF  iawa }  iabb  wieanoe }bm }}nami  b}}i }nbmoniXebb nbbmbbonmaabn}woiwmwo}F oaFFn}oF neb i}b}iab be }eia  mwiebwm eb}  bnb nkannni}oaboa}meoomo}ianebonn FFi bowimbe wimmneaawmaawn Fibniaaa ee F eoioeoioien aanbni   niebemweoai}abmowo }ii mon  aibn}e aibb a}  oaaom niwonnine}eab  kmam}oabo }mi e}mabi namnabmamemwiweboabnoFnibin }ne}naFainiio}}mnimm}ee n}wneibkm}nob ekniowwioobbiFembFeFa nni}aoanena biwbbnF m meeeeana am } kekn  wam kkeoam}m}oaak ooeFFm mne a}}mi}mknio ba a}ooa 

VALIDATION STATS:                                  loss: 2.73938,       accuracy: 0.19320

0%                                        Training on next 50 batches                                        100%
600000 (epoch 0) john_new.txt │ mum jon be pokime. ini inia si │     i  ii iiii        iii  ii  │ loss: 2.40483
600030 (epoch 0) john_new.txt │  na piki ani biari jin torukir │    iiii ii  ii    iii iiii     │ loss: 2.52016
600060 (epoch 0) john_new.txt │  barama nwose fiye fime - ani  │ ii           iii  iii  i ii  i │ loss: 2.65309
600090 (epoch 0) john_new.txt │ a bebe, ani pele sin se ikeli  │  ii    iii iii  ii  ii iii   i │ loss: 2.47526
600120 (epoch 0) john_new.txt │  tamuno be-e, aizik tamuno be- │  ii     i    iiii  iii     i   │ loss: 2.95573
600150 (epoch 0) john_new.txt │ i na sime warasiki bu, ini oko │  ii ii   ii       ii  ii  iiii │ loss: 2.44418
600180 (epoch 0) john_new.txt │ gboribo soni piri ma, se o nwo │        i    iiii ii  ii iiiii  │ loss: 2.51788
600210 (epo

VALIDATION STATS:                                  loss: 2.62395,       accuracy: 0.23837

0%                                        Training on next 50 batches                                        100%
900000 (epoch 0) matt_new.txt │ bobia oku se o diebia. anikiri │ e e  iru oe i iik e   oni i i  │ loss: 1.93331
900030 (epoch 0) matt_new.txt │  mostad dipiyemgbolu kabo mi   │ ni eo ubi i e aae e oa o oa ni │ loss: 2.81876
900060 (epoch 0) matt_new.txt │   da na   nyengi na osi, se da │  iikoa b iae ao ba bri  ie iik │ loss: 2.10299
900090 (epoch 0) matt_new.txt │  o pirime aniatibi ye se o die │ b biki a nni  a i ie ne i iik  │ loss: 1.61312
900120 (epoch 0) matt_new.txt │ na mesaya be devid furobo-e  f │   ii e e ae oirinibi i o e mii │ loss: 2.57856
900150 (epoch 0) matt_new.txt │ w pirime. gbori barafisa owubo │ obi i a  aoori be i a i i o e  │ loss: 2.00760
900180 (epoch 0) matt_new.txt │ y siki la. o bo la se tomoni i │ ebe i ma  o ie oa me iomi i bn │ loss: 2.05782
900210 (epo

VALIDATION STATS:                                  loss: 1.94028,       accuracy: 0.40261

┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
a,r.V*]k,ki aniabaapuaki menamia ienwen oru pirimian oku puri tamini tariri. se nw si serekemi abuna apu bo okoko tore du okine ani kiko ona bere nambe, tumoabu oria be ibume a paake bera men ini akini irikeme oru daru a sike, omi bere omi siri suse in isi aki mi nwoluku o iami iami, aniate swerakume nwo ike nworime ma baa a ane kiri m abuaa kiko mi nwo on ini man iye an ike mamunose ikere,, amn, orikaraba sa ana sari be ganasi amu mi o ia obu siki niama ibeka ii ori kakobu ia oku ba oninbi ma,si iapu anime, i ibi siki inamama ka se mano ki onororibe ini okukume muni, ani mi ori be ane ala maa tinina sonokari puriki, ikumebi ani o inia ko ma ma ini ta broma oni sako ma seki, piri ono bie ni se sama ko nwo ba,n taremo a inase i ori mi na se ne ma siki o ni kurona mane berimia pirase nw

...

VALIDATION STATS:                                  loss: 1.05870,       accuracy: 0.66677

┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
 bereni mi na ini o seni o piri be na iri na omie nwo be soni, ini ineda ineda a se i oki se ini o belemame. aniatibi o bi be na, ini ini na siki ngisi, ani ibi bere nwose oria buchuaye ma bie so mun siki, ori o bereni o pirime nwo be soni, ini nwo beme, nde ani o nemike  a piki nemime na i nyana ye ma mie be, a nemike na i nemime na i berenike  	 jizos be nwose ini beme, i bara i buo ma bie sime ye mi, omine ominea bu ma piribia bara. ani ini oria obudukoapu ma be dapu nwo beme, o beri nyana bo be o nwo beme,   nyanabo be o bi be na i bie sime apu ma na ini i se firimame. o bime o be o pirime na ini o bari i belema inibo dumo pakumabome. aniatibi ini o bara bime, o nemikase i se firimame, o piki i bie sime apu ma nwo beme  i bi ye mi i nwo duko o piri be omine i belema oku mi  a duko

VALIDATION STATS:                                  loss: 1.05554,       accuracy: 0.66819

0%                                        Training on next 50 batches                                        100%
18600000 (epoch 5) luke_8_john_new.txt │ ini iku kiri ogono koro dipiye │ ni iniaburi skono soromsiki em │ loss: 1.21989
18600030 (epoch 5) luke_8_john_new.txt │ bu ibime. bejuadapu ma be diey │ u mni e  oeres apu ma ie tapye │ loss: 1.27518
18600060 (epoch 5) luke_8_john_new.txt │ e, ngisi bin eka fini mi. [ an │ . aaisi oea bne birj ni  a ani │ loss: 1.58949
18600090 (epoch 5) luke_8_john_new.txt │   jizos be pekereme, min oloko │ i izos be nikereme, oin m oko  │ loss: 0.71746
18600120 (epoch 5) luke_8_john_new.txt │ e iru mi na mamapura ololo mi  │  oni bi ba kima u a okolo mi n │ loss: 1.19482
18600150 (epoch 5) luke_8_john_new.txt │ wose o piri nde o miese bugbir │ o e o birimsae o giese oe bari │ loss: 0.99118
18600180 (epoch 5) luke_8_john_new.txt │  isiapu piki nwo beme na,  r

VALIDATION STATS:                                  loss: 1.05551,       accuracy: 0.66905

0%                                        Training on next 50 batches                                        100%
18900000 (epoch 5) luke_8_john_new.txt │  ari bipitoluma se seniabe. an │ bni soei oku a si ia i pe  ani │ loss: 1.32067
18900030 (epoch 5) luke_8_john_new.txt │ ma no mieme.  se jizos be piki │ a nww ie e    e aizos be neki  │ loss: 1.04476
18900060 (epoch 5) luke_8_john_new.txt │ ko bara bu. min mi gose, owubo │ o bara bu  ain mi mose  a ubo  │ loss: 0.65586
18900090 (epoch 5) luke_8_john_new.txt │ mi biebele bu jizos be poki si │ i bie ele su oizos be neki sem │ loss: 0.86124
18900120 (epoch 5) luke_8_john_new.txt │ e ye barachua nweni piri muno, │  ye mura hua iyoni miribban    │ loss: 1.02252
18900150 (epoch 5) luke_8_john_new.txt │  ereso mi, ani wasama mun nyo  │ ore a mi  oni ia ama bin syo b │ loss: 1.40915
18900180 (epoch 5) luke_8_john_new.txt │  fie ka apu, okimun tamuno a

VALIDATION STATS:                                  loss: 1.05290,       accuracy: 0.66931

┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
a na, piki oloko mi na bu boro tamuno be belemame.       a piki nemikase oloko mi na bu bele apu na bereni na nyanabo be na nwengi some.                   be ani mi bu, o mun ma a nwo nyaname, aniatibi ori inia bu sarama se o se biekoroma na ini ini o duko borosam, o bereni bu berenibia oku  o nemikase inibo dabo be na gboriye mie bu, ori o bari fina se ini duko o piriabe na ori o bara bubia siki mi lasam. okuma, omine ineda berekon inibo piri ia oku o nwo beme,   bereni i piri i piri i piribia bara, a nemikase omonibo belemabia bara  a duko o piriabe i bara bime, omine i damabo be bereni bu i bubalabala se o piri bo goyegoye inibo ineda bara in inibo piribia bara, ani ini ori oribia oku. o bereni bo goyegoye inibo tamuno be berenibia bara  a duko o piriabe, a bereni bo goyegoye inibo

VALIDATION STATS:                                  loss: 1.05474,       accuracy: 0.66799

0%                                        Training on next 50 batches                                        100%